# 社群媒體分析 (Betweeness Computing)

In [3]:
import json
import numpy as np
with open('starwars-full-interactions-allCharacters.json') as f:
    data = json.load(f)

### Boolean Matrix

In [4]:
vertex = data['nodes']
edges = data['links']

Matrix = np.zeros((112, 112))

for i in range(len(edges)):
    source = edges[i]['source']
    target = edges[i]['target']
    Matrix[source, target] = 1
    Matrix[target, source] = 1

### Finde Shortest Paths

In [5]:
from typing import List
from sys import maxsize
from collections import deque
 
def add_edge(adj: List[List[int]], src: int, dest: int) -> None:
    adj[src].append(dest)
    adj[dest].append(src)
 
# Function which finds all the paths
# and stores it in paths array
def find_paths(paths: List[List[int]], path: List[int],
               parent: List[List[int]], n: int, u: int) -> None:
    # Base Case
    if (u == -1):
        paths.append(path.copy())
        return
 
    # Loop for all the parents
    # of the given vertex
    for par in parent[u]:
 
        # Insert the current
        # vertex in path
        path.append(u)
 
        # Recursive call for its parent
        find_paths(paths, path, parent, n, par)
 
        # Remove the current vertex
        path.pop()
 

def bfs(adj: List[List[int]], parent: List[List[int]], n: int, start: int) -> None:
    # dist will contain shortest distance
    # from start to every other vertex
    dist = [maxsize for _ in range(n)]
    q = deque()
 
    # Insert source vertex in queue and make
    # its parent -1 and distance 0
    q.append(start)
    parent[start] = [-1]
    dist[start] = 0
 
    # Until Queue is empty
    while q:
        u = q[0]
        q.popleft()
        for v in adj[u]:
            if (dist[v] > dist[u] + 1):
 
                # A shorter distance is found
                # So erase all the previous parents
                # and insert new parent u in parent[v]
                dist[v] = dist[u] + 1
                q.append(v)
                parent[v].clear()
                parent[v].append(u)
 
            elif (dist[v] == dist[u] + 1):
 
                # Another candidate parent for
                # shortes path found
                parent[v].append(u)

            
def print_paths(adj: List[List[int]], n: int, start: int, end: int, sp) -> None:
    paths = []
    path = []
    parent = [[] for _ in range(n)]
 
    # Function call to bfs
    bfs(adj, parent, n, start)
 
    # Function call to find_paths
    find_paths(paths, path, parent, n, end)
    for v in paths:
 
        # Since paths contain each path in reverse order, so reverse it
        v = reversed(v)
 
        # Print node for the current path
        sps = []
        for u in v:
            # print(u, end = " ")
            sps.append(u)
        sp.append(sps)

    return sp
 
if __name__ == "__main__":
    n = 112
    adj = [[] for _ in range(n)]
    for node in edges:
        add_edge(adj, node['source'], node['target'])  

    SP_com = []
    for src in range(112):
        sp = []
        for dest in range(112):
            sp = print_paths(adj, n, src, dest, sp)
        SP_com.append(sp)

### Calculate SPs for each nodes

In [6]:
# 計算 shortest path for each node
Mat_SP = np.zeros((112, 112))

# Mat(i, j) 表示 node i to node j 共有幾條 shortest path
for i in range(112):
    LIST = SP_com[i]
    already_count = 0
    current_count = 0
    # print(LIST)
    for j in range(112):
        if len(LIST) == 1:
            break
        if i == j:
            Mat_SP[i, j] = 0
            current_count += 1
            already_count = current_count
            continue
        while LIST[current_count][-1] == j:
            current_count += 1
            if current_count == len(LIST):
                break
        Mat_SP[i, j] = current_count - already_count
        already_count = current_count

### Calculate Betweeness

In [7]:
# 針對每一個點一開始的 B = 0 
BT = []
for i in range(112):
    betweeness = 0
    # 針對每一個起點  s
    for s in range(112):
        if s == i:
            continue
        SPfroms = SP_com[s]
        
        # 針對都沒有和任何人相連的點
        if len(SPfroms) == 1:
            continue
        
        cur = 0 # 用 cur 控制現在算到 SPfroms 的第幾個 item 
        # 針對每一個終點 t
        for t in range(112):
            # print(cur)
            count = 0 # 用來計算 (s, t) 的 SP 中經過i的次數
            end = SPfroms[cur][-1] # 取最後一個點
            
            # i 不可以為最後一個點
            if t == i:
                while end == i:
                    cur += 1
                    if cur == len(SPfroms):
                        break
                    end = SPfroms[cur][-1]
                continue
            
            while end == t and cur < len(SPfroms):
                if i in SPfroms[cur]:
                    count += 1
                cur += 1
                if cur == len(SPfroms):
                    break
                end = SPfroms[cur][-1]
            
            # 累加(s, p) 的 betweeness
            if Mat_SP[s, t] != 0:
                betweeness += count / Mat_SP[s, t] 
                # print(betweeness)
    BT.append(round(betweeness, 2))

In [9]:
import pandas as pd
df = pd.DataFrame({
    'node': [x for x in range(112)],
    'Betweeness': BT})

In [10]:
rank = df.sort_values(by=['Betweeness'], ascending = False)
top10 = rank[:10]

# Answer and Export as File

In [14]:
MAT_DF = pd.DataFrame(Matrix)
MAT_DF.to_csv("matrix.csv", index = False, header = False)

Rank_DF = top10
Rank_DF.to_csv("rank.csv", index = False, header = False)

In [12]:
pd.DataFrame(Matrix)

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [15]:
top10

,node,Betweeness
7,7,2506.42
24,24,2052.55
20,20,1933.60
67,67,1729.95
73,73,1177.00
66,66,1040.04
11,11,910.91
94,94,892.75
17,17,831.81
1,1,747.96


In [193]:
for i in top10['node'][:10]:
    print(vertex[i]['name'])

OBI-WAN
C-3PO
ANAKIN
LUKE
HAN
DARTH VADER
EMPEROR
POE
PADME
CHEWBACCA
